# Lab 2 Assignment

__Name:__

In the assignment, you will analyze which factors are associated with the number of likes a Facebook post gets. The dataset is from https://archive.ics.uci.edu/ml/datasets/Facebook+metrics. All posts are from the page of a renowned cosmetics brand.

The dependent variable is `like`.

In [99]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [100]:
fb_like = pd.read_csv('http://fengmai.net/download/data/bia652/dataset_Facebook.csv', sep=";")
# remove unneeded variables
not_needed = ['Page total likes', 'Total Interactions']
fb_like = fb_like.drop(not_needed, axis = 1)

In [101]:
fb_like.head(5)

,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share
0,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0
1,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0
2,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0
3,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0
4,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0


## Q1 - Modeling with categorical Xs

- Use [pd.get_dummies()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) to create dummy variables for the following variables: `Type`, `Category`, `Post Month`, `Post Weekday`, `Post Hour`. Make sure that `drop_firt=True` to avoid multicollinearity problem.
- You may want to read [this post](https://stackoverflow.com/questions/24109779/running-get-dummies-on-several-dataframe-columns) to create dummies for multiple columns.
- The origninal columns will be automatically dropped after you create the dummies. 
- You may want to check if any rows have missing variables and drop them accordingly (use [dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)). 
- Create a regression model that uses all the variables (other than like) as independent variables. You'll need to use `statsmodels` and not `sklearn`. Display the OLS Regression Results.  
- No need to split the data to train-test sets for this exercise. 

In [102]:
# Creating a dummy variables for Type, Category, Post Month, Post Weekday, Post Hour

fb_like = pd.get_dummies(data = fb_like,columns=['Type','Category','Post Month', 'Post Weekday', 'Post Hour'], drop_first=True)

In [103]:
fb_like.head()

,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Type_Photo,Type_Status,Type_Video,Category_2,Category_3,Post Month_2,Post Month_3,Post Month_4,Post Month_5,Post Month_6,Post Month_7,Post Month_8,Post Month_9,Post Month_10,Post Month_11,Post Month_12,Post Weekday_2,Post Weekday_3,Post Weekday_4,Post Weekday_5,Post Weekday_6,Post Weekday_7,Post Hour_2,Post Hour_3,Post Hour_4,Post Hour_5,Post Hour_6,Post Hour_7,Post Hour_8,Post Hour_9,Post Hour_10,Post Hour_11,Post Hour_12,Post Hour_13,Post Hour_14,Post Hour_15,Post Hour_16,Post Hour_17,Post Hour_18,Post Hour_19,Post Hour_20,Post Hour_22,Post Hour_23
0,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [104]:
# Checking for NULL values

fb_like.isnull().sum().sum()

6

In [105]:
# drop NULL values
fb_like.dropna(inplace=True)

In [106]:
# Making sure all NULL values are droped
fb_like.isnull().sum().sum()

0

In [107]:
fb_like.shape

(495, 55)

In [108]:
# Separating independent and dependent variables

X = fb_like.loc[:,fb_like.columns != 'like']
X = sm.add_constant(X)
Y = fb_like['like']

In [109]:
X.shape

(495, 55)

In [110]:
Y.head()

0      79.0
1     130.0
2      66.0
3    1572.0
4     325.0
Name: like, dtype: float64

In [111]:
# Applting OLS using statsmodes

OLS_model = sm.OLS(Y,X) 
OLS_results = OLS_model.fit()

In [112]:
# Displaying results
OLS_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   like   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     937.5
Date:                Sun, 15 Nov 2020   Prob (F-statistic):               0.00
Time:                        23:13:34   Log-Likelihood:                -2387.3
No. Observations:                 495   AIC:                             4885.
Df Residuals:                     440   BIC:                             5116.
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================================
                                                                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------
const                                                                  -5.9270     19.916     -0.298      0.766     -45.070      33.216
Paid                                                                    0.2966      3.424      0.087      0.931      -6.434       7.027
Lifetime Post Total Reach                                              -0.0005      0.000     -1.492      0.136      -0.001       0.000
Lifetime Post Total Impressions                                      6.861e-05      0.000      0.618      0.537      -0.000       0.000
Lifetime Engaged Users                                                  0.9812      0.016     62.388      0.000       0.950       1.012
Lifetime Post Consumers                                                -0.9818      0.015    -64.367      0.000      -1.012      -0.952
Lifetime Post Consumptions                                              0.0048      0.001      4.185      0.000       0.003       0.007
Lifetime Post Impressions by people who have liked your Page        -6.614e-05      0.000     -0.556      0.579      -0.000       0.000
Lifetime Post reach by people who like your Page                        0.0010      0.001      1.576      0.116      -0.000       0.002
Lifetime People who have liked your Page and engaged with your post     0.0341      0.007      4.762      0.000       0.020       0.048
comment                                                                 0.8360      0.152      5.488      0.000       0.537       1.135
share                                                                   1.1576      0.095     12.178      0.000       0.971       1.344
Type_Photo                                                            -10.9487      7.716     -1.419      0.157     -26.113       4.215
Type_Status                                                           -37.9881     11.181     -3.398      0.001     -59.963     -16.013
Type_Video                                                              3.6095     15.242      0.237      0.813     -26.346      33.565
Category_2                                                             10.4000      4.783      2.175      0.030       1.000      19.800
Category_3                                                             11.3567      4.333      2.621      0.009       2.841      19.872
Post Month_2                                                           13.4233     10.214      1.314      0.189      -6.650      33.497
Post Month_3                                                           10.0742     10.468      0.962      0.336     -10.500      30.648
Post Month_4                                                           10.9776      8.696  

# Q2 Bootstrap confidence interval

Your tasks is to use the boostrap method to construct a 95% CI for the coefficients for **comment** and **share** using the same model as in Q1. Do you get similar CI to Q1? Why or why not?

In [113]:
from tqdm.auto import tqdm
import statsmodels.formula.api as smf

In [114]:
bootstrapped_ests = []
for i in tqdm(range(5000)):
    # YOUR CODE HERE 
    fb_like_data_boot = fb_like.sample(n=len(fb_like), replace=True)
    fb_like_model_boot = smf.ols(formula='like ~ comment + share', data=fb_like_data_boot).fit()
    bootstrapped_ests.append(fb_like_model_boot.params[['comment', 'share']])
    
b_comments, b_shares = zip(*bootstrapped_ests)

print("The 95% bootstrapped CI of b_comments is [{:.2f}, {:.2f}].".format(
    np.percentile(b_comments, 2.5),
    np.percentile(b_comments, 97.5)))

print("The 95% bootstrapped CI of b_shares is [{:.2f}, {:.2f}].".format(
    np.percentile(b_shares, 2.5),
    np.percentile(b_shares, 97.5)))


The 95% bootstrapped CI of b_comments is [2.10, 5.46].
The 95% bootstrapped CI of b_shares is [4.71, 7.00].


**No we do not get same CI as Q1.** 
*Folloing are the reasons on why they differ.*


1.   CI calculated using Bootstrap is more robust compared to OLS as Bootstrap relay on fewer assumptions than OLS
2.   We use data samples drawn from the original dataset not the actual data set.




## Q3 - Use AIC/BIC for model selection

- Read the [example here](https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html#sphx-glr-auto-examples-linear-model-plot-lasso-model-selection-py) and the [manual here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsIC.html#sklearn.linear_model.LassoLarsIC). You may want to create a separate notebook to run the example.
- Use `LassoLarsIC` to select the best model using the **`bic`** criterion. 
- There is no need to split the data to train-test sets for this exercise. 
- Which X variables are selected by the `LassoLarsIC` method? Answer using variable names. 

In [115]:
X = fb_like.loc[:,fb_like.columns != 'like']
Y = fb_like['like']

In [116]:
Y

0        79.0
1       130.0
2        66.0
3      1572.0
4       325.0
        ...  
494     125.0
495      53.0
496      53.0
497      93.0
498      91.0
Name: like, Length: 495, dtype: float64

In [117]:
#### YOUR CODE HERE ####
from sklearn import linear_model
from sklearn.linear_model import LassoLarsIC

In [118]:
model_LassoLarsIC = LassoLarsIC(criterion='bic')
model_LassoLarsIC.fit(X,Y)

LassoLarsIC(copy_X=True, criterion='bic', eps=2.220446049250313e-16,
            fit_intercept=True, max_iter=500, normalize=True, positive=False,
            precompute='auto', verbose=False)

In [119]:
model_LassoLarsIC.coef_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00814194, 0.        , 2.10037166,
       4.71135015, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [120]:
# X variables which are selected by the LassoLarsIC method
X.columns[np.abs(model_LassoLarsIC.coef_) > 0]

Index(['Lifetime Post reach by people who like your Page', 'comment', 'share'], dtype='object')

## Q4 - Compare out-of-sample perormance

- Split the data to 90% train and 10% test set. 
- Train three models using the train set:
   * `LassoLarsIC` model (a model that uses BIC to select variables)
   * a full model (a model that includes all X variables)
   * a `LassoCV` model (a model that uses CV on train set to select variables)
- Remember to train a scaler using train set only, and transform both X_train and X_test. 
- Compare their test-set performance using MSE. Which model has the best performance? 

In [121]:
mse = dict()

In [122]:
#### YOUR CODE HERE ####
from sklearn.model_selection import train_test_split

In [123]:
# Spliting data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state = 1)

In [124]:
print(X_train.shape,X_test.shape)

(445, 54) (50, 54)


In [125]:
print(y_train.shape,y_test.shape)

(445,) (50,)


In [126]:
from sklearn import preprocessing

In [127]:
scaler = preprocessing.StandardScaler().fit(X_train)

In [128]:
# Linear modes using LassoLarasIC
model_1 = LassoLarsIC(criterion='bic')
model_1.fit(scaler.transform(X_train),y_train)

LassoLarsIC(copy_X=True, criterion='bic', eps=2.220446049250313e-16,
            fit_intercept=True, max_iter=500, normalize=True, positive=False,
            precompute='auto', verbose=False)

In [129]:
model1_yhat = model_1.predict(scaler.transform((X_test)))

In [130]:
from sklearn import metrics

In [131]:
mse['LassoLarsIC'] = metrics.mean_squared_error(y_true=y_test,y_pred=model1_yhat)

In [132]:
# Linear model using all Xs
from sklearn.linear_model import LinearRegression

In [133]:
model_2 = LinearRegression()
model_2.fit(scaler.transform(X_train),y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [134]:
model2_yhat = model_2.predict(scaler.transform((X_test)))

In [135]:
mse['Full Model'] = metrics.mean_squared_error(y_true=y_test,y_pred=model2_yhat)

In [136]:
from sklearn.linear_model import Lasso, LassoCV, lars_path

In [137]:
# Linear model using LassoCV
model_3 = Lasso(alpha = 0.1, normalize=True)
model_3.fit(scaler.transform(X_train),y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000, normalize=True,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)

In [138]:
model3_yhat = model_3.predict(scaler.transform((X_test)))

In [139]:
mse['LassoCV'] = metrics.mean_squared_error(y_true=y_test,y_pred=model3_yhat)

In [140]:
# Comparing all MSEs
tmp = min(mse.values())
res = [key for key in mse if mse[key] == tmp]

In [141]:
print('Model with minimum MSE is',res,'And its MSE is',tmp)

Model with minimum MSE is ['LassoCV'] And its MSE is 2700.020879952409
